In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

full_df = pd.read_csv('brooklyn_2022_sales_updated.csv')
high_low_df = pd.read_csv('highest_three_lowest_four_updated.csv')
batt_park_df = pd.read_csv('Batt_park_2022_sales_updated.csv')
last_zip = pd.read_csv('zip_10069_updated.csv')

frames = [full_df, high_low_df, batt_park_df, last_zip]
full_df = pd.concat(frames)
full_df.columns
full_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'highest_three_lowest_four_updated.csv'

In [ ]:
#create full DF and combined columns for first and second mtgs into one colums, added a count columns
#remove columns that have LTV of over 100
full_df = full_df.fillna(0)
full_df['Amount Mortgaged'] = full_df['1st Amount'] 
full_df['Count'] = 1
full_df = full_df.loc[:, ['Count', 'ZIP', 'Type', 'Owner Occ?', 
                                 'Purchase Amt', 'Amount Mortgaged']]


In [ ]:
full_df.shape

In [ ]:
full_df['% Borrowed'] = full_df['Amount Mortgaged']/ full_df['Purchase Amt']*100
full_df = full_df[full_df['% Borrowed'] < 100]
full_df.shape

In [ ]:
#calculate how much mortgage tax was collected.
# for mortgages less than $500,000, tax is 1.8% for mortgages of $500,000, 1.95% 
#calculate the mansion tax as 1% for sales over $1,000,000

full_df['Mtg Tx Pd'] = np.where(full_df['Amount Mortgaged'] >= 500000, 
                                 full_df['Amount Mortgaged'] * 0.0195, full_df['Amount Mortgaged'] * 0.018)
full_df['Mansion Tax Pd'] = np.where(full_df['Purchase Amt'] >= 1000000, full_df['Purchase Amt'] * 0.01, 0)
full_df.head()

In [ ]:
#created two dataframes, one for investor purchases and the other for primary residences
investor_df = full_df[(full_df['Owner Occ?'] ==1)]
owner_occ_df = full_df[(full_df['Owner Occ?'] == 0)]


In [ ]:
#calculate new sales tax info for private residences, 0.9% for purchases under $1mil and 
#calculate new sales tax for investment purchases, 4.4357%
owner_occ_df['New Sales Tax Amount'] = np.where(owner_occ_df['Purchase Amt'] >= 1000000, owner_occ_df['Purchase Amt'] * 0.009,
                                owner_occ_df['Purchase Amt'] * 0.014)
investor_df['New Sales Tax Amount'] = investor_df['Purchase Amt'] * 0.04438

owner_occ_df.tail()

In [ ]:
#show the total mtg tax and mansion taxes collected and compare to what our hypothetical tax would have raised

total_mtg_tax_pd = owner_occ_df['Mtg Tx Pd'].sum()
total_mansion_tax_pd = owner_occ_df['Mansion Tax Pd'].sum()
total_est_tax = owner_occ_df['New Sales Tax Amount'].sum()
total_mtg_tax_inves_pd = investor_df['Mtg Tx Pd'].sum()
total_mansion_tax_inves_pd = investor_df['Mansion Tax Pd'].sum()
total_est_inves_tax = investor_df['New Sales Tax Amount'].sum()
print(f"The total mtg tax paid in bklyn by individuals purchasing primary residences in 2022 was ${total_mtg_tax_pd:,.0f} and the total mansion tax collected was ${total_mansion_tax_pd:,.0f}.")
print(f'The combined mtg and mansion tax was {total_mtg_tax_pd + total_mansion_tax_pd:,.0f}, our hypothetical sales tax would have raised ${total_est_tax:,.0f}')
print(f"The total mtg tax paid in bklyn by investors in 2022 was ${total_mtg_tax_inves_pd:,.0f} and the total mansion tax collected was ${total_mansion_tax_inves_pd:,.0f}.")
print(f'The combined mtg and mansion tax was {total_mtg_tax_inves_pd + total_mansion_tax_inves_pd:,.0f}, our hypothetical sales tax on investments would have raised ${total_est_inves_tax:,.0f}')
print(f'The total tax raised in 2022 was {total_mtg_tax_pd + total_mansion_tax_pd + total_mtg_tax_inves_pd + total_mansion_tax_inves_pd:,.0f} and the total amount that would have been raised would have been ${total_est_tax + total_est_inves_tax:,.0f}')

In [ ]:
owner_occ_by_zip = owner_occ_df.groupby(['ZIP']).agg({'Count': 'sum', 'Purchase Amt': 'sum', 
                                                            '% Borrowed': 'mean',
                                                            'Amount Mortgaged': 'sum', 
                                                            'Mtg Tx Pd': 'sum', 'Mansion Tax Pd': 'sum', 
                                                            'New Sales Tax Amount': 'sum'})

owner_occ_by_zip.head()

In [ ]:
investor_by_zip = investor_df.groupby(['ZIP']).agg({'Count': 'sum', 'Purchase Amt': 'sum', 
                                                            '% Borrowed': 'mean',
                                                            'Amount Mortgaged': 'sum', 
                                                            'Mtg Tx Pd': 'sum', 'Mansion Tax Pd': 'sum', 
                                                            'New Sales Tax Amount': 'sum'})
investor_by_zip.shape

In [ ]:
owner_occ_by_zip['Average Purchase $'] = owner_occ_by_zip['Purchase Amt']/owner_occ_by_zip['Count']
owner_occ_by_zip['Average Mtg'] = owner_occ_by_zip['Amount Mortgaged']/owner_occ_by_zip['Count']
owner_occ_by_zip['Ave Hypothetical Tax'] = owner_occ_by_zip['New Sales Tax Amount']/owner_occ_by_zip['Count']
pd.options.display.float_format = '{:.1f}'.format

owner_occ_by_zip.head()

In [ ]:
investor_by_zip['Average Purchase $'] = investor_by_zip['Purchase Amt']/investor_by_zip['Count']
investor_by_zip['Average Mtg'] = investor_by_zip['Amount Mortgaged']/investor_by_zip['Count']
investor_by_zip['Ave Hypothetical Tax'] = investor_by_zip['New Sales Tax Amount']/investor_by_zip['Count']
pd.options.display.float_format = '{:.1f}'.format

investor_by_zip.head()

In [ ]:
frames = [owner_occ_df, investor_df]
calculated_full_df = pd.concat(frames)
calculated_full_df.shape

In [ ]:
calculated_full_df.to_csv('Sales_info_with_taxes.csv')